# 12 - Matrix Inverse
- 12.1 Concepts and applications
- 12.2 Inverse of a diagonal matrix
- 12.3 Inverse of a $2 \times 2$ matrix
- 12.4 MCA Algorithm
- 12.5 Inverse via row reduction
- 12.6 Left inverse for tall matrices
- 12.7 Right inverse for wide matrices
- 12.8 Pseudoinverse, part 1
- 12.9 Exercises
- 12.10 Answers
- 12.11 Coding challenges
- 12.12 Code solutions

Notes, code snippets, and the end of chapter exercises from the book _Linear Algebra: Theory, Intuition, Code_ by Mike X Cohen. 

Find more information about the book on [github](https://github.com/mikexcohen/LinAlgBook) and [amazon](https://www.amazon.com/Linear-Algebra-Theory-Intuition-Code/dp/9083136604).

In [1]:
import numpy as np
import sympy as sym

## 12.1 Concepts and applications
Inverse is the matrix analog of the multiplicative inverse (division).

$$
A A^{-1} = A^{-1} A = I
$$

Inverse of the inverse is the original matrix.

$$
(A^{-1})^{-1} = A
$$

Transpose of the inverse is equal to the inverse of the transpose. Notation for this is $A^{-T}$.

$$
(A^{-1})^{T} = (A^{T})^{-1} = A^{-T}
$$

Note
- Matrix inverse exists if and only if matrix is square and full rank.
- Inverse of a square matrix, if it exists, is the same whether the inverse pre-multiplies the matrix on the left or post-multiplies the matrix on the right.
- Inverse of a matrix is unique.
- Similar to determinants (_section 11.9 Theory vs. Practice_) explicitly computing the inverse is not computationally useful.
    - Most problems that require the inverse in theory are solved without explicitly computing the inverse.

### Inverse of Symmetric Matrices
Inverse of a symmetric matrix is itself symmetric. If $A = A^T$ then $A^{-1} = A^{-T}$.

### Methods for Computing Inverse
1. MCA aka Cramer's Rule (_section 12.4_)
2. RREF, row-reduced echelon form reduction (_section 12.5_)
3. SVD, singular value decomposition (_chapter 16_)

## 12.2 Inverse of a diagonal matrix
Inverse of a diagonal matrix is equal to the reciprocal of the diagonal elements.

Example
$$
A =
\begin{bmatrix}
d_1 & 0 & 0 \\
0 & d_2 & 0 \\
0 & 0 & d_3 \\
\end{bmatrix} \quad
A^{-1} = 
\begin{bmatrix}
\frac{1}{d_1} & 0 & 0 \\
0 & \frac{1}{d_2} & 0 \\
0 & 0 & \frac{1}{d_3} \\
\end{bmatrix}
$$

Note
- If any of the diagonal elements are 0, then the matrix must be singular and inverse does not exist.

## 12.3 Inverse of a $2 \times 2$ matrix
Inverse of a $2 \times 2$ matrix is given as a special case of Cramer's Rule.

$$
A = 
\begin{bmatrix}
a & b \\
c & d \\
\end{bmatrix} \quad
A^{-1} = 
\frac{1}{ad - bc}
\begin{bmatrix}
d & -b \\
-c & a \\
\end{bmatrix} \quad
$$

Note
- If the $\text{det}(A) = 0$, then the matrix must be singular and inverse does not exist.

## 12.4 MCA Algorithm
MCA algorithm stands for minors, cofactors, and adjoint is used to find the inverse of a matrix and is also known as _Cramer's Rule_.

The adjoint matrix of $A$ is the inverse $A^{-1}$.

1. Compute the minors matrix $M$.
- Element $m_{i,j}$ of $M$ equals the determinant of the submatrix formed by excluding the _ith_ row and _jth_ column of $A$.

2. Compute the cofactors matrix $C$.
- Cofactors matrix $C$ equal to the Hadamard product of chessboard matrix $G$ and minors matrix $M$.
$$
C = G \odot M
$$
- Chessboard matrix $G$ is a matrix of alternating $\pm 1$ with $g_{i,j} = (-1)^{i+j}$ with upper left hand corner starting from $+1$.

3. Compute the adjoint matrix $A^{-1}$.
- Adjoint matrix $A^{-1}$ is equal to the transpose of the cofactors matrix multiplied by the reciprocal of the determinant of the original matrix.
$$
A^{-1} = \frac{1}{\text{det}(A)} C^T
$$

## 12.5 Inverse via row reduction

### RREF Inverse
Inverse of a matrix $A$ can be obtained via RREF in a manner analogous to Gauss-Jordan elimination.

1. Augment $A$ with $I$ to produce the matrix $[ A | I ]$
2. Use elementary row operations to transform augmented matrix $\text{RREF}([ A | I ]) = [ I | A^{-1} ]$

Notes
- By comparison Gauss-Jordan is used to solve $Ax = b$ for $x$ and the approach to find the inverse solves $A A^{-1} = I$ for $A^{-1}$. 

In [2]:
n = 5
A = np.random.random((n, n))
I = np.eye(n)

# Augment A and I to produce a single matrix [A|I].
AI = np.hstack((A, I))

# Compute RREF(A|I) = [I|A^{-1}].
rrefAI, _ = sym.Matrix(AI).rref()
Ainv = np.array(rrefAI[:,n:])

# Verify A A^{-1} = I.
np.testing.assert_almost_equal(A @ Ainv, np.eye(n))

# Verify the inverse computed via RREF equals the library for returning the inverse.
np.testing.assert_almost_equal(Ainv, np.linalg.inv(A))

## 12.6 Left inverse for tall matrices
Tall matrices $T$ do not have a two-sided inverse, but they have a one-sided left inverse if $\text{rank}(T) = n$.

Left inverse of a tall matrix $T$ is given by the product of 3 matrices.

$$
(T^T T)^{-1} T^T T = I
$$

Notes
- Tall matrices are rectangular, non-square, and have more rows than columns.
- Product $T^T T$ is square $n \times n$ and invertible if $\text{rank}(T) = n$ where $n$ is the number of columns.
- Left inverse is used in overdetermined systems eg systems with more equations than unknowns aka _least squares_.

In [3]:
# Verify that the product of the left inverse of a 
# tall full-column-rank matrix and itself is the identity matrix.
m, n = 7, 5
T = np.random.random((m, n))

Tleftinv = np.linalg.inv(T.T @ T) @ T.T

np.testing.assert_almost_equal(Tleftinv @ T, np.eye(n))

## 12.7 Right inverse for wide matrices
Wide matrices $W$ do not have a two-sided inverse, but they can have a one-sided right inverse if $\text{rank}(W) = m$.

Right inverse of a wide matrix $W$ is given by the product of 3 matrices.

$$
W W^T (W W^T)^{-1} = I
$$

Notes
- Wide matrices are rectangular, non-square, and have more columns than rows.
- Product $W W^T$ is square $m \times m$ and invertible if $\text{rank}(W) = m$ where $m$ is the number of rows.

In [4]:
# Verify that the product of the right inverse of a 
# wide full-row-rank matrix and itself is the identity matrix.
m, n = 5, 7
W = np.random.random((m, n))

Wrightinv = W.T @ np.linalg.inv(W @ W.T)

np.testing.assert_almost_equal(W @ Wrightinv, np.eye(m))

## 12.8 Pseudoinverse, part 1

Notes
- Pseudoinverse exists even when the matrix is singular.
- Pseudoinverse exists even when the matrix is not square.
- Pseudoinverse is the same as the inverse for full-rank square matrices.
- Notation used for pseudoinverse varies $A^\dagger, A^*, A^+$ are in common use.
- For tall full-column-rank matrices $A^+ = (T^T T)^{-1} T^T$ (_left-sided inverse_).
- For wide full-row-rank matrices $A^+ = W^T (W W^T)^{-1}$ (_right-sided inverse_).

## 12.11 Coding challenges

> 1. Implement the MCA algorithm to compute the matrix inverse. Test your algorithm by computing the inverse of random matrices and compare against the library function for returning the inverse.

In [5]:
def minors(A):
    """
    minors returns the minors matrix of A

    :param A: numpy.ndarray  Matrix A
    :return: numpy.ndarray   Matrix M
    """
    assert A.shape[0] == A.shape[1], "A is not square"

    M, n = np.zeros_like(A), A.shape[0]
    for i in range(n):
        idx_i = np.r_[:i:,i+1:n:]
        for j in range(n):
            idx_j = np.r_[:j:,j+1:n:]
            # Compute the determinant of the submatrix formed by 
            # excluding row i and column j of the matrix A.
            M[i,j] = np.linalg.det(A[np.ix_(idx_i, idx_j)])
    return M


def chessboard(m, n):
    """
    chessboard returns the matrix G of +/- 1 values according to $g_{i,j} = (-1)^{i+j}$

    :param m: int            Number of rows.
    :param n: int            Number of columns.
    :return: numpy.ndarray   Matrix G
    """
    G = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            G[i,j] = (-1)**(i+j)
    return G


def adjoint(A):
    """
    adjoint returns the inverse of A using MCA algorithm aka Cramer's Rule

    :param A: numpy.ndarray  Matrix A
    :return: numpy.ndarray   Matrix Ainv
    """
    assert A.shape[0] == A.shape[1], "A is not square"

    # Compute the determinant of A.
    detA = np.linalg.det(A)
    assert detA != 0., "A is singular"

    # Compute the cofactors matrix.
    C = chessboard(A.shape[0], A.shape[0]) * minors(A)

    return 1./detA * C.T


n = 5
A = np.random.random((n, n))
Ainv = adjoint(A)

# Verify A A^{-1} = I.
np.testing.assert_almost_equal(A @ Ainv, np.eye(n))

# Verify the adjoint equals the inverse.
np.testing.assert_almost_equal(Ainv, np.linalg.inv(A))

> 2. Show that (1) the pseudoinverse is the same as the real inverse for a full-rank square matrix; and (2) the pseudoinverse is the same as the left inverse for a full-column-rank tall matrix.

In [6]:
# Verify the p-inv is the same as the real inverse for a full-rank square matrix.
n = 5
A = np.random.random((n, n))

Ainv = np.linalg.inv(A)
Apinv = np.linalg.pinv(A)

np.testing.assert_almost_equal(Ainv, Apinv)

# Verify the p-inv is the same as the left inverse for a full-column-rank tall matrix.
m, n = 7, 5
T = np.random.random((m, n))

Tleftinv = np.linalg.inv(T.T @ T) @ T.T
Tpinv = np.linalg.pinv(T)

np.testing.assert_almost_equal(Tleftinv, Tpinv)